In [12]:
#import root_numpy
#from root_pandas import read_root
import pandas as pd
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import uproot

from functools import partial
import glob
from tqdm import tqdm
from multiprocessing import Pool


In [2]:
! ls /data2/tracking/trackingGG/dump_15114101/ | head

DownstreamDumper_runNb_10221103_evtNb_201.root
DownstreamDumper_runNb_10221103_evtNb_202.root
DownstreamDumper_runNb_10221103_evtNb_203.root
DownstreamDumper_runNb_10221103_evtNb_204.root
DownstreamDumper_runNb_10221103_evtNb_205.root
DownstreamDumper_runNb_10221103_evtNb_206.root
DownstreamDumper_runNb_10221103_evtNb_207.root
DownstreamDumper_runNb_10221103_evtNb_208.root
DownstreamDumper_runNb_10221103_evtNb_209.root
DownstreamDumper_runNb_10221103_evtNb_210.root
ls: write error: Broken pipe


In [3]:
true_track_ntuples = glob.glob("/data2/tracking/trackingGG/dump_15114101/*.root")

In [4]:
len(true_track_ntuples)

73969

In [5]:
event = uproot.open("/data2/tracking/trackingGG/dump_15114101/DownstreamDumper_runNb_10221103_evtNb_201.root:Particle_Data")

In [6]:
event.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
UT_cos               | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
UT_cosT              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
UT_dxDy              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
UT_lhcbID            | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
UT_planeCode         | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
UT_sinT              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
UT_tanT              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
UT_weight            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
UT_xAtYEq0           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
UT_xAtYMid           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
UT_xT                | std::

In [7]:
def filter_branches(branch, requested_pattern, excluded_patterns):
    """This helper function filter a branches that we would like to conver to pandas 
    Args:
      requested_pattern: a list of pattern that has to be present in the branch name;
      excluded_patterns: a list of patterns that indicates a given branch has to be excluded. The exclusion is superior to requested
                         e.g exclided_patterns ["foo"], requested_patterns ["bar"] branch -> "foobar" -> false 
    """
    branch_name = branch.name
    if any(map(lambda pattern: pattern in branch_name, excluded_patterns)): return False
    return any(map(lambda pattern: pattern in branch_name, requested_pattern))

In [8]:
requested = ["track","has","is","eventID","key"]
excluded = ["state","residue","ghostProbability"]

event_df = event.arrays(filter_branch= lambda  branch: partial(filter_branches,requested_pattern=requested,excluded_patterns=excluded)(branch), library="pd")

In [9]:
event_df

,track_charge,track_chi2,track_chi2PerDoF,track_nLHCbIDs,track_p,track_phi,track_position_phi,track_position_r,track_position_x,track_position_y,...,eventID,particle_key,particle_hasScifi,particle_hasUT,particle_hasVelo,particle_isDown,particle_isDown_noVelo,particle_isLong,particle_isLong_andUT,partiole_isElectron
0,1.0,1.278146,0.182592,12.0,23744.048551,-2.696049,-2.331667,275.657777,-190.080475,-199.641235,...,201,2035,True,True,True,True,False,True,True,False
1,1.0,4.993319,0.713331,12.0,9216.070642,2.611930,2.220800,872.014687,-527.733826,694.194946,...,201,2082,True,True,True,True,False,True,True,False
2,-1.0,3.866762,0.552395,12.0,18153.844322,0.894309,1.409535,516.864430,82.989517,510.158386,...,201,2083,True,True,True,True,False,True,True,False
3,1.0,1.232309,0.205385,11.0,12769.130712,2.131943,1.818912,1409.637625,-346.176331,1366.469971,...,201,2145,True,True,True,True,False,True,True,False
4,1.0,0.323443,0.053907,11.0,6437.135788,2.653733,2.308582,1448.264033,-974.176453,1071.657104,...,201,2152,True,True,True,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,1.0,7.206513,1.201085,11.0,1702.830303,3.062398,2.400955,1139.246049,-840.807007,768.716553,...,201,9304,True,True,True,True,False,True,True,False
157,-1.0,2.095205,0.419041,10.0,6544.255724,-0.109341,-0.413063,378.102723,346.302643,-151.776642,...,201,9352,True,True,True,True,False,True,True,False
158,-1.0,0.585813,0.097635,11.0,7216.476591,-0.287957,-0.601960,759.690409,626.157715,-430.181396,...,201,9407,True,True,True,True,False,True,True,False
159,1.0,2.221262,0.317323,12.0,2919.751377,3.092438,2.985373,1147.140007,-1133.170776,178.477417,...,201,9411,True,True,False,True,True,False,False,True


In [41]:
def convert_root_to_pandas(file_path):
    event = uproot.open("{}:Particle_Data".format(file_path))
    
    requested = ["track","has","is","eventID","key"]
    excluded = ["state","residue","ghostProbability"]
    try:
        return event.arrays(filter_branch= lambda  branch: partial(filter_branches,requested_pattern=requested,excluded_patterns=excluded)(branch), library="pd")
    except e:
        print(e)
        return None

In [18]:
%%time 
with Pool(41) as p:
    events_df = pd.concat(list(tqdm(p.imap(convert_root_to_pandas, true_track_ntuples), total=len(true_track_ntuples))))


100%|██████████| 73969/73969 [11:02<00:00, 111.70it/s]


CPU times: user 9min 53s, sys: 44.5 s, total: 10min 38s
Wall time: 12min 20s


In [19]:
events_df

,track_charge,track_chi2,track_chi2PerDoF,track_nLHCbIDs,track_p,track_phi,track_position_phi,track_position_r,track_position_x,track_position_y,...,eventID,particle_key,particle_hasScifi,particle_hasUT,particle_hasVelo,particle_isDown,particle_isDown_noVelo,particle_isLong,particle_isLong_andUT,partiole_isElectron
0,1.0,2.261579,0.376930,11.0,4923.881506,-3.115572,-3.072188,603.199306,-601.747070,-41.831402,...,159651,2222,True,False,False,False,False,False,False,True
1,-1.0,6.342810,0.906116,12.0,2159.286580,-0.006293,-0.041903,1145.680471,1144.674805,-47.993053,...,159651,2224,True,False,False,False,False,False,False,True
2,-1.0,7.558596,1.259766,11.0,1953.560581,-0.024234,-0.155405,1514.672557,1496.419067,-234.441742,...,159651,2242,True,True,True,True,False,True,True,True
3,1.0,4.356195,0.726032,11.0,3825.724500,3.122918,2.986581,579.261462,-572.315918,89.433388,...,159651,2255,True,True,False,True,True,False,False,False
4,-1.0,3.676422,0.525203,12.0,1799.832644,0.115020,0.578678,1265.392218,1059.369507,692.064819,...,159651,2265,True,True,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1.0,1.458956,0.243159,11.0,3006.985696,-2.728876,-1.683146,1137.733041,-127.555580,-1130.560059,...,170523,7084,True,True,True,True,False,True,True,False
101,1.0,2.559061,0.365580,12.0,3785.883868,-2.344596,-0.973824,1245.145505,699.947205,-1029.786987,...,170523,7149,True,True,True,True,False,True,True,False
102,1.0,2.179678,0.363280,11.0,6405.086185,-3.004129,-2.689266,329.013314,-295.925385,-143.798218,...,170523,7185,True,False,False,False,False,False,False,False
103,1.0,2.236754,0.372792,11.0,4506.041970,2.868708,2.632723,631.677427,-551.641357,307.746948,...,170523,7192,True,False,False,False,False,False,False,False


In [20]:
events_df.to_parquet("/data2/tracking/tracking_upgrade_ad/data/true_no_hits_15114101.parquet")

In [52]:
events_df.filter(regex="particle*").sum()/len(events_df)

particle_key              6610.234482
particle_hasScifi            0.992350
particle_hasUT               0.717370
particle_hasVelo             0.616141
particle_isDown              0.713730
particle_isDown_noVelo       0.136763
particle_isLong              0.612475
particle_isLong_andUT        0.576967
dtype: float64

In [49]:
len(events_df)

10568908

In [64]:
len(events_df)*0.13

1373958.04

In [63]:
events_df.filter(regex="particle_*").sum()

particle_key              6.986296e+10
particle_hasScifi         1.048806e+07
particle_hasUT            7.581816e+06
particle_hasVelo          6.511941e+06
particle_isDown           7.543347e+06
particle_isDown_noVelo    1.445433e+06
particle_isLong           6.473189e+06
particle_isLong_andUT     6.097914e+06
dtype: float64

## Convert ghosts

In [26]:
! ls /data2/tracking/trackingGG/

BrunelDev_v52r6
BrunelDev_v60r0
DaVinciDev_v42r0
KsPiPi2016Down_baseline_.root
MC_Upgrade_Beam7000GeVUpgradeMagUpNu7.625nsPythia8_Sim10Up03OldP8Tuning_RecoUp02_11104142_XDST
Pattern_Creator
Pattern_Creator_New
README
README-seed-clf
Seed Classifier performance analysis.tar.gz
Seed_clf_performance_analysis
all.zip
baseline_build
brunel_build_v61r0
cp_dump.sh
data_dump
dataraw
davinci_data_output
dirac_dump
dirac_dump2
downstream_dumper
downstream_dumper.tar.gz
downstream_dumper2
dump_11154100
dump_15114101
dump_15144103
dump_15144103_ghost
dump_build
dump_small_18_02_20
dumper2
dumper3
gccbuild
kernel_estim
mcdata
moore_build
new_dump.root
olga
pc.tar.gz
pca_data.csv
pca_weights.csv
probki_danych
single_event
single_event2
single_pattern
temp
test_xdst.xdst
upgrade


In [42]:
%%time 
ghosts_path = glob.glob("/data2/tracking/trackingGG/dump_15144103_ghost/*.root")
with Pool(41) as p:
    ghosts_df = pd.concat(list(tqdm(p.imap(convert_root_to_pandas, ghosts_path[:15465]), total=len(ghosts_path[:15465]))))


100%|██████████| 15465/15465 [01:43<00:00, 149.33it/s]


CPU times: user 1min 33s, sys: 16 s, total: 1min 49s
Wall time: 1min 57s


In [44]:
ghosts_df

,track_charge,track_chi2,track_chi2PerDoF,track_nLHCbIDs,track_p,track_phi,track_position_phi,track_position_r,track_position_x,track_position_y,track_position_z,track_pt,track_tx,track_ty,track_pseudoRapidity
0,-1.0,14.173387,2.834677,12.0,1708.679041,0.663611,1.869638,1191.614841,-350.827520,1138.800238,7500.0,441.725063,0.210821,0.164842,2.028792
1,1.0,7.862321,1.572464,11.0,50005.988946,1.180322,1.713843,2176.096505,-310.223879,2153.870271,7500.0,15132.959213,0.120853,0.293611,1.864691
2,1.0,21.830846,5.457711,11.0,23253.552775,2.842934,2.299620,188.481940,-125.527615,140.599643,7500.0,1672.736552,-0.068929,0.021221,3.323848
3,1.0,14.112966,3.528242,11.0,75890.537134,2.945978,2.481008,150.940004,-119.187290,92.613577,7500.0,5104.724648,-0.066131,0.013104,3.391140
4,-1.0,13.164444,2.632889,11.0,10458.477796,-0.036632,0.307578,74.972363,71.453877,22.697989,7500.0,1325.713753,0.127705,-0.004680,2.754568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,1.0,7.767622,2.589207,9.0,3606.455460,3.120202,-3.093675,731.091743,-730.252582,-35.018613,7500.0,1163.786987,-0.340857,0.007292,1.797080
8,-1.0,4.976554,1.658851,9.0,31915.134880,-0.394808,-2.104135,786.029615,-399.626481,-676.861309,7500.0,11284.555682,0.348918,-0.145389,1.699962
9,-1.0,7.693757,2.564586,9.0,16912.698360,0.015030,-2.848377,107.161725,-102.588000,-30.973174,7500.0,3921.360297,0.238327,0.003582,2.141054
10,1.0,1.770942,0.885471,9.0,413.445749,-2.061473,-1.113748,1782.909880,786.800112,-1599.910380,7500.0,134.616341,-0.162271,-0.303731,1.787621


In [45]:
ghosts_df.to_parquet("/data2/tracking/tracking_upgrade_ad/data/ghosts_no_hits_15144103.parquet")

In [27]:
event = uproot.open("/data2/tracking/trackingGG/dump_15144103_ghost/GhostDumper_runNb_10088608_evtNb_702.root:Particle_Data")

In [29]:
event.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
track_charge         | double                   | AsDtype('>f8')
track_chi2           | double                   | AsDtype('>f8')
track_chi2PerDoF     | double                   | AsDtype('>f8')
track_nLHCbIDs       | double                   | AsDtype('>f8')
track_p              | double                   | AsDtype('>f8')
track_phi            | double                   | AsDtype('>f8')
track_position_phi   | double                   | AsDtype('>f8')
track_position_r     | double                   | AsDtype('>f8')
track_position_x     | double                   | AsDtype('>f8')
track_position_y     | double                   | AsDtype('>f8')
track_position_z     | double                   | AsDtype('>f8')
track_pt             | double                   | AsDtype('>f8')
track_tx             | double                   | AsDtype(